In [1]:
import pandas as pd

"""
Calculate your teaching hours.
"""
data = pd.read_csv("TimeEdit_Ezra_Eisbrenner_2024-01-04_11_53.csv", header=2)
largescale = data.Course.unique()[1]
mesoscale = data.Course.unique()[2]

# Graded reports is the number of reports you have to grade if there was 1 student.
# Time for grading does not scale with number of students,
# as per decision by the department. You don't get hours for pass-fail evaluation.
courses = {
    largescale: {"first_time": False, "n_graded_reports": 3},
    mesoscale: {"first_time": True, "n_graded_reports": 0},
}


# ==================================================================================== #
# No change needed below here!
# Unless code is broken.
# ==================================================================================== #

# TODO add a check if a long session goes over lunch and remove an hour
# TODO ask if things like course development and presentations get a first time multiplier too

factors = {
    "Computer lab.": 2,
    "Laboratory experiment": 2,
    "Presentation": 1,
    "Course development": 1,
    "Calculation exercise": 4,
}
first_time_multiplier = 1.5

for t in ["Begin", "End"]:
    data[t] = data.apply(
        lambda r: pd.Timestamp(" ".join([r[f"{t} date"], r[f"{t} time"]])), axis=1
    )
data["Begin"] = data["Begin"].apply(
    lambda x: x - pd.Timedelta(minutes=15 if x.minute == 15 else 0)
)
data["duration"] = data["End"] - data["Begin"]

for course in courses:
    course_data = data.loc[lambda r: r.Course == course]

    break_later = False
    for activity in course_data.Activities:
        if activity not in factors.keys():
            print(f"{activity} has no assigned factor.")
            break_later = True
    if break_later:
        raise KeyError("Code breaks because there is an unknown activity.")

    totaltime = 0
    print(f"{course}\n")
    for activity, factor in factors.items():
        time = course_data.loc[
            lambda r: r.Activities == activity
        ].duration.sum().total_seconds() / (60 * 60)
        print(f"{activity}: {time} x {factor}")
        totaltime = totaltime + time * factor

    print(f'Report grading: {float(courses[course]["n_graded_reports"])} x 4')
    totaltime = totaltime + courses[course]["n_graded_reports"] * 4

    if courses[course]["first_time"]:
        totaltime = totaltime * first_time_multiplier
    print(f'First time: {courses[course]["first_time"]}')
    print(f"Time in total: {totaltime} hours\n")

MO8004. Modelling of Large-Scale Circulation in Atmosphere and Ocean

Computer lab.: 65.0 x 2
Laboratory experiment: 8.0 x 2
Presentation: 6.0 x 1
Course development: 1.0 x 1
Calculation exercise: 0.0 x 4
Report grading: 3.0 x 4
First time: False
Time in total: 165.0 hours

MO8005. Mesoscale Meteorology

Computer lab.: 10.0 x 2
Laboratory experiment: 0.0 x 2
Presentation: 4.0 x 1
Course development: 0.0 x 1
Calculation exercise: 6.0 x 4
Report grading: 0.0 x 4
First time: True
Time in total: 72.0 hours

